In [1]:

# Init
# choropleth
from __future__ import division
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import os
import matplotlib.pyplot as plt
import seaborn as sns

# County level data retrieval

In [6]:
# # df_19 = pd.read_excel('./raw_data/2019_acs_block_group_data.xlsx', na_values=".", skiprows=[0])
df_19 = pd.read_csv('./raw_data/ny_2019_acs_block_group_data.csv', na_values=".", skiprows=[0])
df_19 = df_19.replace(np.nan, 0)
df_19.to_csv('./raw_data/ny_2019_acs_block_group_data_zeroed.csv')
df_19 = pd.read_csv('./raw_data/ny_2019_acs_block_group_data_zeroed.csv')


In [12]:
df_19

,Unnamed: 0,Geo_FIPS,Geo_NAME,Geo_QName,Geo_STUSAB,Geo_SUMLEV,Geo_GEOCOMP,Geo_FILEID,Geo_LOGRECNO,Geo_US,...,ACS19_5yr_B23025004s,ACS19_5yr_B23025005s,ACS19_5yr_B23025006s,ACS19_5yr_B23025007s,PCT_ACS19_5yr_B23025002,PCT_ACS19_5yr_B23025003,PCT_ACS19_5yr_B23025004,PCT_ACS19_5yr_B23025005,PCT_ACS19_5yr_B23025006,PCT_ACS19_5yr_B23025007
0,0,10010201001,Block Group 1,"Block Group 1, Census Tract 201, Autauga Count...",al,150,0,ACSSF,2953,0.0,...,63.03030,6.060606,7.272727,86.06061,49.38,49.38,47.99,1.39,0.00,50.62
1,1,10010201002,Block Group 2,"Block Group 2, Census Tract 201, Autauga Count...",al,150,0,ACSSF,2954,0.0,...,98.18182,9.696970,7.272727,70.30303,68.11,68.11,66.15,1.96,0.00,31.89
2,2,10010202001,Block Group 1,"Block Group 1, Census Tract 202, Autauga Count...",al,150,0,ACSSF,2955,0.0,...,52.72727,7.272727,7.878788,67.27273,44.20,42.52,42.52,0.00,1.68,55.80
3,3,10010202002,Block Group 2,"Block Group 2, Census Tract 202, Autauga Count...",al,150,0,ACSSF,2956,0.0,...,60.00000,15.757580,7.272727,65.45454,51.33,51.33,47.96,3.37,0.00,48.67
4,4,10010203001,Block Group 1,"Block Group 1, Census Tract 203, Autauga Count...",al,150,0,ACSSF,2957,0.0,...,137.57580,29.090910,5.454545,102.42420,68.39,68.17,65.55,2.61,0.23,31.61
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220328,220328,721537506011,Block Group 1,"Block Group 1, Census Tract 7506.01, Yauco Mun...",pr,150,0,ACSSF,6111,0.0,...,40.00000,48.484850,8.484848,123.03030,24.80,24.80,9.73,15.07,0.00,75.20
220329,220329,721537506012,Block Group 2,"Block Group 2, Census Tract 7506.01, Yauco Mun...",pr,150,0,ACSSF,6112,0.0,...,128.48480,81.818180,8.484848,176.36360,42.12,42.12,33.48,8.64,0.00,57.88
220330,220330,721537506013,Block Group 3,"Block Group 3, Census Tract 7506.01, Yauco Mun...",pr,150,0,ACSSF,6113,0.0,...,72.72727,8.484848,8.484848,136.36360,44.33,44.33,44.33,0.00,0.00,55.67
220331,220331,721537506021,Block Group 1,"Block Group 1, Census Tract 7506.02, Yauco Mun...",pr,150,0,ACSSF,6114,0.0,...,163.03030,20.606060,8.484848,213.33330,28.78,28.78,27.25,1.53,0.00,71.22


In [4]:
# 2019 Data Cleanup

# ACS19_5yr_B01001001 - total_population
df = pd.DataFrame()
# df['fips'] = df_19['Geo_FIPS']
df['zip'] = df_19['Geo_ZCTA5']

df['total_population'] = df_19['ACS19_5yr_B01001001']
df['log_total_population'] = np.log(df['total_population'])
df['population_density'] = df_19['ACS19_5yr_B01001001'] / df_19['Geo_AREALAND']

# Workers 16 Years and Over in Households,ACS19_5yr_B08014001
df['total_workers'] = df_19['ACS19_5yr_B08014001']

df['land_area'] = df_19['Geo_AREALAND']# / 1000000 # convert to sqkm
df['sqrt_land_area'] = np.sqrt(df_19['Geo_AREALAND']) # / 1000000 # convert to sqkm

# "Workers Whose Means of Transportation Is Car, Truck, or Van: Aggregate Number of Vehicles (Car, Truck, or Van) Used in Commuting",ACS19_5yr_B08015001
df['aggregate_num_vehicles'] = df_19['ACS19_5yr_B08015001']

# aggregate_time_per_worker
# Workers 16 Years and Over Who Did Not Work At Home: Aggregate Travel Time to Work (in Minutes).4,ACS19_5yr_B08136001
df['aggregate_time_commute'] = df_19['ACS19_5yr_B08136001']

# "% Workers 16 Years and Over Who Did Not Work At Home: Aggregate Travel Time to Work (in Minutes): Car, Truck, or Van - Drove Alone",ACS19_5yr_C08136002
df['aggregate_time_commute_drive_alone'] = df_19['ACS19_5yr_B08136002']

# "% Workers 16 Years and Over Who Did Not Work At Home: Aggregate Travel Time to Work (in Minutes): Car, Truck, or Van - Carpooled",ACS19_5yr_C08136003
df['aggregate_time_commute_carpool'] = df_19['ACS19_5yr_B08136003']

# % Workers 16 Years and Over Who Did Not Work At Home: Aggregate Travel Time to Work (in Minutes): Public Transportation (Excluding Taxicab),ACS19_5yr_C08136004
df['aggregate_time_commute_public_transit'] = df_19['ACS19_5yr_B08136004']

# "% Workers 16 Years and Over Who Did Not Work At Home: Aggregate Travel Time to Work (in Minutes): Taxicab, Motorcycle, Bicycle, Walked, or Other Means",ACS19_5yr_C08136005
df['aggregate_time_commute_other_means'] = df_19['ACS19_5yr_B08136005']

# "Workers 16 Years and Over Who Did Not Work At Home: Car, Truck, or Van - Drove Alone.1",ACS19_5yr_C08134011
df['total_workers_drove_alone'] = df_19['ACS19_5yr_B08134011']

# "Workers 16 Years and Over Who Did Not Work At Home: Car, Truck, or Van - Carpooled.1",ACS19_5yr_C08134021
df['total_workers_carpooled'] = df_19['ACS19_5yr_B08134021']

# Workers 16 Years and Over Who Did Not Work At Home: Public Transportation (Excluding Taxicab).1,ACS19_5yr_C08134031
df['total_workers_public_transit'] = df_19['ACS19_5yr_B08134031']

# "Workers 16 Years and Over Who Did Not Work At Home: Taxicab, Motorcycle, Bicycle,  Walked, or Other Means",ACS19_5yr_C08134041
df['total_workers_other_means'] = df_19['ACS19_5yr_B08134041']

# Total.181,ACS19_5yr_B08134001
df['ttl_wrkr_cmute'] = df_19['ACS19_5yr_B08134001']


# Car Truck Or Van - Drove Alone.13,ACS19_5yr_B08122005
df['wrkrs_drove_alone'] = df_19['ACS19_5yr_B08122005']
# Car Truck Or Van - Drove Alone: Below 100 Percent Of The Poverty Level,ACS19_5yr_B08122006
df['wrkrs_drove_alone_below_100_poverty'] = df_19['ACS19_5yr_B08122006']
df['pct_wrkrs_drove_alone_below_100_poverty'] = df_19['ACS19_5yr_B08122006'] / df['ttl_wrkr_cmute']
# Car Truck Or Van - Drove Alone: 100 To 149 Percent Of The Poverty Level,ACS19_5yr_B08122007
df['wrkrs_drove_alone_100_to_150_poverty'] = df_19['ACS19_5yr_B08122007']
# Car Truck Or Van - Drove Alone: At Or Above 150 Percent Of The Poverty Level,ACS19_5yr_B08122008
df['wrkrs_drove_alone_over_150_poverty'] = df_19['ACS19_5yr_B08122008']

# Car Truck Or Van - Carpooled.13,ACS19_5yr_B08122009
df['wrkrs_carpool'] = df_19['ACS19_5yr_B08122009']
# Car Truck Or Van - Carpooled: Below 100 Percent Of The Poverty Level,ACS19_5yr_B08122010
df['wrkrs_carpool_below_100_poverty'] = df_19['ACS19_5yr_B08122010']
df['pct_wrkrs_carpool_below_100_poverty'] = df_19['ACS19_5yr_B08122010'] / df['ttl_wrkr_cmute']
# Car Truck Or Van - Carpooled: 100 To 149 Percent Of The Poverty Level,ACS19_5yr_B08122011
df['wrkrs_carpool_100_to_150_poverty'] = df_19['ACS19_5yr_B08122011']
# Car Truck Or Van - Carpooled: At Or Above 150 Percent Of The Poverty Level,ACS19_5yr_B08122012
df['wrkrs_carpool_over_150_poverty'] = df_19['ACS19_5yr_B08122012']

# Public Transportation (Excluding Taxicab).14,ACS19_5yr_B08122013
df['wrkrs_public_transit'] = df_19['ACS19_5yr_B08122013']
# Public Transportation (Excluding Taxicab): Below 100 Percent Of The Poverty Level,ACS19_5yr_B08122014
df['wrkrs_public_transit_below_100_poverty'] = df_19['ACS19_5yr_B08122014']
df['pct_wrkrs_public_transit_below_100_poverty'] = df_19['ACS19_5yr_B08122014'] / df['ttl_wrkr_cmute']
# Public Transportation (Excluding Taxicab): 100 To 149 Percent Of The Poverty Level,ACS19_5yr_B08122015
df['wrkrs_public_transit_100_to_150_poverty'] = df_19['ACS19_5yr_B08122015']
# Public Transportation (Excluding Taxicab): At Or Above 150 Percent Of The Poverty Level,ACS19_5yr_B08122016
df['wrkrs_public_transit_over_150_poverty'] = df_19['ACS19_5yr_B08122016']

# Walked.14,ACS19_5yr_B08122017
df['wrkrs_walked'] = df_19['ACS19_5yr_B08122017']
# Walked: Below 100 Percent Of The Poverty Level,ACS19_5yr_B08122018
df['wrkrs_walked_below_100_poverty'] = df_19['ACS19_5yr_B08122018']
df['pct_wrkrs_walked_below_100_poverty'] = df_19['ACS19_5yr_B08122018'] / df['ttl_wrkr_cmute']
# Walked: 100 To 149 Percent Of The Poverty Level,ACS19_5yr_B08122019
df['wrkrs_walked_100_to_150_poverty'] = df_19['ACS19_5yr_B08122019']
# Walked: At Or Above 150 Percent Of The Poverty Level,ACS19_5yr_B08122020
df['wrkrs_walked_over_150_poverty'] = df_19['ACS19_5yr_B08122020']

# Taxicab Motorcycle Bicycle Or Other Means.13,ACS19_5yr_B08122021
df['wrkrs_commute_other_means'] = df_19['ACS19_5yr_B08122021']
# Taxicab Motorcycle Bicycle Or Other Means: Below 100 Percent Of The Poverty Level,ACS19_5yr_B08122022
df['wrkrs_commute_other_means_below_100_poverty'] = df_19['ACS19_5yr_B08122022']
df['pct_wrkrs_commute_other_means_below_100_poverty'] = df_19['ACS19_5yr_B08122022'] / df['ttl_wrkr_cmute']
# Taxicab Motorcycle Bicycle Or Other Means: 100 To 149 Percent Of The Poverty Level,ACS19_5yr_B08122023
df['wrkrs_commute_other_means_100_to_150_poverty'] = df_19['ACS19_5yr_B08122023']
# Taxicab Motorcycle Bicycle Or Other Means: At Or Above 150 Percent Of The Poverty Level,ACS19_5yr_B08122024
df['wrkrs_commute_other_means_over_150_poverty'] = df_19['ACS19_5yr_B08122024']

# # Worked From Home.14,ACS19_5yr_B08122025
# df['total_workers_other_means'] = df_19['ACS19_5yr_B08134041']
# # Worked From Home Below 100 Percent Of The Poverty Level,ACS19_5yr_B08122026
# df['total_workers_other_means'] = df_19['ACS19_5yr_B08134041']
# # Worked From Home 100 To 149 Percent Of The Poverty Level,ACS19_5yr_B08122027
# df['total_workers_other_means'] = df_19['ACS19_5yr_B08134041']
# # Worked From Home At Or Above 150 Percent Of The Poverty Level,ACS19_5yr_B08122028
# df['total_workers_other_means'] = df_19['ACS19_5yr_B08134041']

# Less Than 10 Minutes,ACS19_5yr_B08134002
df['ttl_wrkr_cmute_under_10_min'] = df_19['ACS19_5yr_B08134002']
# 10 To 14 Minutes.1,ACS19_5yr_B08134003
df['ttl_wrkr_cmute_10_to_14_min'] = df_19['ACS19_5yr_B08134003']
# 15 To 19 Minutes.1,ACS19_5yr_B08134004
df['ttl_wrkr_cmute_15_to_14_min'] = df_19['ACS19_5yr_B08134004']
# 20 To 24 Minutes.1,ACS19_5yr_B08134005
df['ttl_wrkr_cmute_20_to_24_min'] = df_19['ACS19_5yr_B08134005']
# 25 To 29 Minutes.1,ACS19_5yr_B08134006
df['ttl_wrkr_cmute_25_to_29_min'] = df_19['ACS19_5yr_B08134006']
# 30 To 34 Minutes.1,ACS19_5yr_B08134007
df['ttl_wrkr_cmute_30_to_34_min'] = df_19['ACS19_5yr_B08134007']
# 35 To 44 Minutes,ACS19_5yr_B08134008
df['ttl_wrkr_cmute_35_to_44_min'] = df_19['ACS19_5yr_B08134008']
# 45 To 59 Minutes.1,ACS19_5yr_B08134009
df['ttl_wrkr_cmute_45_to_59_min'] = df_19['ACS19_5yr_B08134009']
# 60 Or More Minutes,ACS19_5yr_B08134010
df['ttl_wrkr_cmute_over_60'] = df_19['ACS19_5yr_B08134010']
# 15 min range Percents
df['pct_wrkr_cmute_01_to_14_min'] = (df['ttl_wrkr_cmute_under_10_min'] + df['ttl_wrkr_cmute_10_to_14_min']) / df['ttl_wrkr_cmute']
df['pct_wrkr_cmute_15_to_29_min'] = (df['ttl_wrkr_cmute_15_to_14_min'] + df['ttl_wrkr_cmute_20_to_24_min'] + df['ttl_wrkr_cmute_25_to_29_min']) / df['ttl_wrkr_cmute']
df['pct_wrkr_cmute_30_to_44_min'] = (df['ttl_wrkr_cmute_30_to_34_min'] + df['ttl_wrkr_cmute_35_to_44_min']) / df['ttl_wrkr_cmute']
df['pct_wrkr_cmute_44_to_59_min'] = df['ttl_wrkr_cmute_45_to_59_min'] / df['ttl_wrkr_cmute']
df['pct_wrkr_cmute_over_60_min'] = df['ttl_wrkr_cmute_over_60'] / df['ttl_wrkr_cmute']
# # 15 min range Percents
# df['pct_wrkr_cmute_01_to_14_min'] = (df['ttl_wrkr_cmute_under_10_min'] + df['ttl_wrkr_cmute_10_to_14_min']) / df['ttl_wrkr_cmute']
# df['pct_wrkr_cmute_15_to_29_min'] = (df['ttl_wrkr_cmute_15_to_14_min'] + df['ttl_wrkr_cmute_20_to_24_min'] + df['ttl_wrkr_cmute_25_to_29_min']) / df['ttl_wrkr_cmute']
# df['pct_wrkr_cmute_30_to_44_min'] = (df['ttl_wrkr_cmute_30_to_34_min'] + df['ttl_wrkr_cmute_35_to_44_min']) / df['ttl_wrkr_cmute']
# df['pct_wrkr_cmute_44_to_59_min'] = df['ttl_wrkr_cmute_45_to_59_min'] / df['ttl_wrkr_cmute']
# df['pct_wrkr_cmute_over_60_min'] = df['ttl_wrkr_cmute_over_60'] / df['ttl_wrkr_cmute']

# Car Truck Or Van.1,ACS19_5yr_B08134011
df['ttl_car_wrkr_cmute'] = df_19['ACS19_5yr_B08134011']
# Car Truck Or Van: Less Than 10 Minutes,ACS19_5yr_B08134012
df['ttl_car_wrkr_cmute_under_10_min'] = df_19['ACS19_5yr_B08134012']
# Car Truck Or Van: 10 To 14 Minutes,ACS19_5yr_B08134013
df['ttl_car_wrkr_cmute_10_to_14_min'] = df_19['ACS19_5yr_B08134013']
# Car Truck Or Van: 15 To 19 Minutes,ACS19_5yr_B08134014
df['ttl_car_wrkr_cmute_15_to_14_min'] = df_19['ACS19_5yr_B08134014']
# Car Truck Or Van: 20 To 24 Minutes,ACS19_5yr_B08134015
df['ttl_car_wrkr_cmute_20_to_24_min'] = df_19['ACS19_5yr_B08134015']
# Car Truck Or Van: 25 To 29 Minutes,ACS19_5yr_B08134016
df['ttl_car_wrkr_cmute_25_to_29_min'] = df_19['ACS19_5yr_B08134016']
# Car Truck Or Van: 30 To 34 Minutes,ACS19_5yr_B08134017
df['ttl_car_wrkr_cmute_30_to_34_min'] = df_19['ACS19_5yr_B08134017']
# Car Truck Or Van: 35 To 44 Minutes,ACS19_5yr_B08134018
df['ttl_car_wrkr_cmute_35_to_44_min'] = df_19['ACS19_5yr_B08134018']
# Car Truck Or Van: 45 To 59 Minutes,ACS19_5yr_B08134019
df['ttl_car_wrkr_cmute_45_to_59_min'] = df_19['ACS19_5yr_B08134019']
# Car Truck Or Van: 60 Or More Minutes,ACS19_5yr_B08134020
df['ttl_car_wrkr_cmute_over_60_min'] = df_19['ACS19_5yr_B08134020']
# # Car Truck Or Van: Drove Alone.1,ACS19_5yr_B08134021
# df['ttl_car_wrkr_cmute'] = df_19['ACS19_5yr_B08134021']
# # Car Truck Or Van: Drove Alone: Less Than 10 Minutes,ACS19_5yr_B08134022
# df['ttl_car_wrkr_cmute_under_10_min'] = df_19['ACS19_5yr_B08134022']
# # Car Truck Or Van: Drove Alone: 10 To 14 Minutes,ACS19_5yr_B08134023
# df['ttl_car_wrkr_cmute_10_to_14_min'] = df_19['ACS19_5yr_B08134023']
# # Car Truck Or Van: Drove Alone: 15 To 19 Minutes,ACS19_5yr_B08134024
# df['ttl_car_wrkr_cmute_15_to_14_min'] = df_19['ACS19_5yr_B08134024']
# # Car Truck Or Van: Drove Alone: 20 To 24 Minutes,ACS19_5yr_B08134025
# df['ttl_car_wrkr_cmute_20_to_24_min'] = df_19['ACS19_5yr_B08134025']
# # Car Truck Or Van: Drove Alone: 25 To 29 Minutes,ACS19_5yr_B08134026
# df['ttl_car_wrkr_cmute_25_to_29_min'] = df_19['ACS19_5yr_B08134026']
# # Car Truck Or Van: Drove Alone: 30 To 34 Minutes,ACS19_5yr_B08134027
# df['ttl_car_wrkr_cmute_30_to_34_min'] = df_19['ACS19_5yr_B08134027']
# # Car Truck Or Van: Drove Alone: 35 To 44 Minutes,ACS19_5yr_B08134028
# df['ttl_car_wrkr_cmute_35_to_44_min'] = df_19['ACS19_5yr_B08134028']
# # Car Truck Or Van: Drove Alone: 45 To 59 Minutes,ACS19_5yr_B08134029
# df['ttl_car_wrkr_cmute_45_to_59_min'] = df_19['ACS19_5yr_B08134029']
# # Car Truck Or Van: Drove Alone: 60 Or More Minutes,ACS19_5yr_B08134030
# df['ttl_car_wrkr_cmute_over_60_min'] = df_19['ACS19_5yr_B08134030']
# 15 min range Percents
df['pct_car_wrkr_cmute'] = df['ttl_car_wrkr_cmute'] / df['ttl_wrkr_cmute']
df['pct_car_wrkr_cmute_01_to_14_min'] = (df['ttl_car_wrkr_cmute_under_10_min'] + df['ttl_car_wrkr_cmute_10_to_14_min']) / df['ttl_wrkr_cmute']
df['pct_car_wrkr_cmute_15_to_29_min'] = (df['ttl_car_wrkr_cmute_15_to_14_min'] + df['ttl_car_wrkr_cmute_20_to_24_min'] + df['ttl_car_wrkr_cmute_25_to_29_min']) / df['ttl_wrkr_cmute']
df['pct_car_wrkr_cmute_30_to_44_min'] = (df['ttl_car_wrkr_cmute_30_to_34_min'] + df['ttl_car_wrkr_cmute_35_to_44_min']) / df['ttl_wrkr_cmute']
df['pct_car_wrkr_cmute_44_to_59_min'] = df['ttl_car_wrkr_cmute_45_to_59_min'] / df['ttl_wrkr_cmute']
df['pct_car_wrkr_cmute_over_60_min'] = df['ttl_car_wrkr_cmute_over_60_min'] / df['ttl_wrkr_cmute']

df['log_pct_car_wrkr_cmute']               = np.log(df['pct_car_wrkr_cmute'])
df['log_pct_car_wrkr_cmute_01_to_14_min']  = np.log(df['pct_car_wrkr_cmute_01_to_14_min'])
df['log_pct_car_wrkr_cmute_15_to_29_min']  = np.log(df['pct_car_wrkr_cmute_15_to_29_min'])
df['log_pct_car_wrkr_cmute_30_to_44_min']  = np.log(df['pct_car_wrkr_cmute_30_to_44_min'])
df['log_pct_car_wrkr_cmute_44_to_59_min']  = np.log(df['pct_car_wrkr_cmute_44_to_59_min'])
df['log_pct_car_wrkr_cmute_over_60_min']   = np.log(df['pct_car_wrkr_cmute_over_60_min'])
# # 15 min range Percents
# df['pct_car_wrkr_cmute_01_to_14_min'] = (df['ttl_car_wrkr_cmute_under_10_min'] + df['ttl_car_wrkr_cmute_10_to_14_min']) / df['ttl_car_wrkr_cmute']
# df['pct_car_wrkr_cmute_15_to_29_min'] = (df['ttl_car_wrkr_cmute_15_to_14_min'] + df['ttl_car_wrkr_cmute_20_to_24_min'] + df['ttl_car_wrkr_cmute_25_to_29_min']) / df['ttl_car_wrkr_cmute']
# df['pct_car_wrkr_cmute_30_to_44_min'] = (df['ttl_car_wrkr_cmute_30_to_34_min'] + df['ttl_car_wrkr_cmute_35_to_44_min']) / df['ttl_car_wrkr_cmute']
# df['pct_car_wrkr_cmute_44_to_59_min'] = df['ttl_car_wrkr_cmute_45_to_59_min'] / df['ttl_car_wrkr_cmute']
# df['pct_car_wrkr_cmute_over_60_min'] = df['ttl_car_wrkr_cmute_over_60_min'] / df['ttl_car_wrkr_cmute']

df['zero_check'] = 1
# Zero Check
df['zero_check'] = df['pct_car_wrkr_cmute_01_to_14_min'] * \
                        df['pct_car_wrkr_cmute_15_to_29_min'] * \
                        df['pct_car_wrkr_cmute_30_to_44_min'] * \
                        df['pct_car_wrkr_cmute_44_to_59_min'] * \
                        df['pct_car_wrkr_cmute_over_60_min'] * df['zero_check']

##### SOLO ABD CARPOOL DRIVIGN IS COMBINED INTO OVERALL CAR

# #  Carpooled,ACS19_5yr_B08134031
# df['ttl_carpool_wrkr_cmute'] = df_19['ACS19_5yr_B08134031']
# #  Carpooled: Less Than 10 Minutes,ACS19_5yr_B08134032
# df['ttl_carpool_wrkr_cmute_under_10_min'] = df_19['ACS19_5yr_B08134032']
# #  Carpooled: 10 To 14 Minutes,ACS19_5yr_B08134033
# df['ttl_carpool_wrkr_cmute_10_to_14_min'] = df_19['ACS19_5yr_B08134033']
# #  Carpooled: 15 To 19 Minutes,ACS19_5yr_B08134034
# df['ttl_carpool_wrkr_cmute_15_to_14_min'] = df_19['ACS19_5yr_B08134034']
# #  Carpooled: 20 To 24 Minutes,ACS19_5yr_B08134035
# df['ttl_carpool_wrkr_cmute_20_to_24_min'] = df_19['ACS19_5yr_B08134035']
# #  Carpooled: 25 To 29 Minutes,ACS19_5yr_B08134036
# df['ttl_carpool_wrkr_cmute_25_to_29_min'] = df_19['ACS19_5yr_B08134036']
# #  Carpooled: 30 To 34 Minutes,ACS19_5yr_B08134037
# df['ttl_carpool_wrkr_cmute_30_to_34_min'] = df_19['ACS19_5yr_B08134037']
# #  Carpooled: 35 To 44 Minutes,ACS19_5yr_B08134038
# df['ttl_carpool_wrkr_cmute_35_to_44_min'] = df_19['ACS19_5yr_B08134038']
# #  Carpooled: 45 To 59 Minutes,ACS19_5yr_B08134039
# df['ttl_carpool_wrkr_cmute_45_to_59_min'] = df_19['ACS19_5yr_B08134039']
# #  Carpooled: 60 Or More Minutes,ACS19_5yr_B08134040
# df['ttl_carpool_wrkr_cmute_over_60_min'] = df_19['ACS19_5yr_B08134040']
# # 15 min range Percents
# df['pct_carpool_wrkr_cmute'] = df['ttl_carpool_wrkr_cmute'] / df['ttl_wrkr_cmute']
# df['pct_carpool_wrkr_cmute_01_to_14_min'] = (df['ttl_carpool_wrkr_cmute_under_10_min'] + df['ttl_carpool_wrkr_cmute_10_to_14_min']) / df['ttl_wrkr_cmute']
# df['pct_carpool_wrkr_cmute_15_to_29_min'] = (df['ttl_carpool_wrkr_cmute_15_to_14_min'] + df['ttl_carpool_wrkr_cmute_20_to_24_min'] + df['ttl_carpool_wrkr_cmute_25_to_29_min']) / df['ttl_wrkr_cmute']
# df['pct_carpool_wrkr_cmute_30_to_44_min'] = (df['ttl_carpool_wrkr_cmute_30_to_34_min'] + df['ttl_carpool_wrkr_cmute_35_to_44_min']) / df['ttl_wrkr_cmute']
# df['pct_carpool_wrkr_cmute_44_to_59_min'] = df['ttl_carpool_wrkr_cmute_45_to_59_min'] / df['ttl_wrkr_cmute']
# df['pct_carpool_wrkr_cmute_over_60_min'] = df['ttl_carpool_wrkr_cmute_over_60_min'] / df['ttl_wrkr_cmute']
# # # 15 min range Percents
# # df['pct_carpool_wrkr_cmute_01_to_14_min'] = (df['ttl_carpool_wrkr_cmute_under_10_min'] + df['ttl_carpool_wrkr_cmute_10_to_14_min']) / df['ttl_carpool_wrkr_cmute']
# # df['pct_carpool_wrkr_cmute_15_to_29_min'] = (df['ttl_carpool_wrkr_cmute_15_to_14_min'] + df['ttl_carpool_wrkr_cmute_20_to_24_min'] + df['ttl_carpool_wrkr_cmute_25_to_29_min']) / df['ttl_carpool_wrkr_cmute']
# # df['pct_carpool_wrkr_cmute_30_to_44_min'] = (df['ttl_carpool_wrkr_cmute_30_to_34_min'] + df['ttl_carpool_wrkr_cmute_35_to_44_min']) / df['ttl_carpool_wrkr_cmute']
# # df['pct_carpool_wrkr_cmute_44_to_59_min'] = df['ttl_carpool_wrkr_cmute_45_to_59_min'] / df['ttl_carpool_wrkr_cmute']
# # df['pct_carpool_wrkr_cmute_over_60_min'] = df['ttl_carpool_wrkr_cmute_over_60_min'] / df['ttl_carpool_wrkr_cmute']


# Public Transportation (Excluding Taxicab).20,ACS19_5yr_B08134061
df['ttl_all_public_tspt_wrkr_cmute'] = df_19['ACS19_5yr_B08134061']
# Public Transportation (Excluding Taxicab): Less Than 10 Minutes,ACS19_5yr_B08134062
df['ttl_all_public_tspt_wrkr_cmute_under_10_min'] = df_19['ACS19_5yr_B08134062']
# Public Transportation (Excluding Taxicab): 10 To 14 Minutes,ACS19_5yr_B08134063
df['ttl_all_public_tspt_wrkr_cmute_10_to_14_min'] = df_19['ACS19_5yr_B08134063']
# Public Transportation (Excluding Taxicab): 15 To 19 Minutes,ACS19_5yr_B08134064
df['ttl_all_public_tspt_wrkr_cmute_15_to_14_min'] = df_19['ACS19_5yr_B08134064']
# Public Transportation (Excluding Taxicab): 20 To 24 Minutes,ACS19_5yr_B08134065
df['ttl_all_public_tspt_wrkr_cmute_20_to_24_min'] = df_19['ACS19_5yr_B08134065']
# Public Transportation (Excluding Taxicab): 25 To 29 Minutes,ACS19_5yr_B08134066
df['ttl_all_public_tspt_wrkr_cmute_25_to_29_min'] = df_19['ACS19_5yr_B08134066']
# Public Transportation (Excluding Taxicab): 30 To 34 Minutes,ACS19_5yr_B08134067
df['ttl_all_public_tspt_wrkr_cmute_30_to_34_min'] = df_19['ACS19_5yr_B08134067']
# Public Transportation (Excluding Taxicab): 35 To 44 Minutes,ACS19_5yr_B08134068
df['ttl_all_public_tspt_wrkr_cmute_35_to_44_min'] = df_19['ACS19_5yr_B08134068']
# Public Transportation (Excluding Taxicab): 45 To 59 Minutes,ACS19_5yr_B08134069
df['ttl_all_public_tspt_wrkr_cmute_45_to_59_min'] = df_19['ACS19_5yr_B08134069']
# Public Transportation (Excluding Taxicab): 60 Or More Minutes,ACS19_5yr_B08134070
df['ttl_all_public_tspt_wrkr_cmute_over_60_min'] = df_19['ACS19_5yr_B08134070']
# 15 min range Percents
df['pct_all_public_tspt_wrkr_cmute'] = df['ttl_all_public_tspt_wrkr_cmute'] / df['ttl_wrkr_cmute']
df['pct_all_public_tspt_wrkr_cmute_01_to_14_min'] = (df['ttl_all_public_tspt_wrkr_cmute_under_10_min'] + df['ttl_all_public_tspt_wrkr_cmute_10_to_14_min']) / df['ttl_wrkr_cmute']
df['pct_all_public_tspt_wrkr_cmute_15_to_29_min'] = (df['ttl_all_public_tspt_wrkr_cmute_15_to_14_min'] + df['ttl_all_public_tspt_wrkr_cmute_20_to_24_min'] + df['ttl_all_public_tspt_wrkr_cmute_25_to_29_min']) / df['ttl_wrkr_cmute']
df['pct_all_public_tspt_wrkr_cmute_30_to_44_min'] = (df['ttl_all_public_tspt_wrkr_cmute_30_to_34_min'] + df['ttl_all_public_tspt_wrkr_cmute_35_to_44_min']) / df['ttl_wrkr_cmute']
df['pct_all_public_tspt_wrkr_cmute_44_to_59_min'] = df['ttl_all_public_tspt_wrkr_cmute_45_to_59_min'] / df['ttl_wrkr_cmute']
df['pct_all_public_tspt_wrkr_cmute_over_60_min'] = df['ttl_all_public_tspt_wrkr_cmute_over_60_min'] / df['ttl_wrkr_cmute']
# # 15 min range Percents
# df['pct_all_public_tspt_wrkr_cmute_01_to_14_min'] = (df['ttl_all_public_tspt_wrkr_cmute_under_10_min'] + df['ttl_all_public_tspt_wrkr_cmute_10_to_14_min']) / df['ttl_all_public_tspt_wrkr_cmute']
# df['pct_all_public_tspt_wrkr_cmute_15_to_29_min'] = (df['ttl_all_public_tspt_wrkr_cmute_15_to_14_min'] + df['ttl_all_public_tspt_wrkr_cmute_20_to_24_min'] + df['ttl_all_public_tspt_wrkr_cmute_25_to_29_min']) / df['ttl_all_public_tspt_wrkr_cmute']
# df['pct_all_public_tspt_wrkr_cmute_30_to_44_min'] = (df['ttl_all_public_tspt_wrkr_cmute_30_to_34_min'] + df['ttl_all_public_tspt_wrkr_cmute_35_to_44_min']) / df['ttl_all_public_tspt_wrkr_cmute']
# df['pct_all_public_tspt_wrkr_cmute_44_to_59_min'] = df['ttl_all_public_tspt_wrkr_cmute_45_to_59_min'] / df['ttl_all_public_tspt_wrkr_cmute']
# df['pct_all_public_tspt_wrkr_cmute_over_60_min'] = df['ttl_all_public_tspt_wrkr_cmute_over_60_min'] / df['ttl_all_public_tspt_wrkr_cmute']
# Zero Check
df['zero_check'] = df['pct_all_public_tspt_wrkr_cmute_01_to_14_min'] * \
                        df['pct_all_public_tspt_wrkr_cmute_15_to_29_min'] * \
                        df['pct_all_public_tspt_wrkr_cmute_30_to_44_min'] * \
                        df['pct_all_public_tspt_wrkr_cmute_44_to_59_min'] * \
                        df['pct_all_public_tspt_wrkr_cmute_over_60_min'] * df['zero_check']


# Public Transportation (Excluding Taxicab): Bus.1,ACS19_5yr_B08134071
df['ttl_bus_wrkr_cmute'] = df_19['ACS19_5yr_B08134071']
# Public Transportation (Excluding Taxicab): Bus: Less Than 10 Minutes,ACS19_5yr_B08134072
df['ttl_bus_wrkr_cmute_under_10_min'] = df_19['ACS19_5yr_B08134072']
# Public Transportation (Excluding Taxicab): Bus: 10 To 14 Minutes,ACS19_5yr_B08134073
df['ttl_bus_wrkr_cmute_10_to_14_min'] = df_19['ACS19_5yr_B08134073']
# Public Transportation (Excluding Taxicab): Bus: 15 To 19 Minutes,ACS19_5yr_B08134074
df['ttl_bus_wrkr_cmute_15_to_14_min'] = df_19['ACS19_5yr_B08134074']
# Public Transportation (Excluding Taxicab): Bus: 20 To 24 Minutes,ACS19_5yr_B08134075
df['ttl_bus_wrkr_cmute_20_to_24_min'] = df_19['ACS19_5yr_B08134075']
# Public Transportation (Excluding Taxicab): Bus: 25 To 29 Minutes,ACS19_5yr_B08134076
df['ttl_bus_wrkr_cmute_25_to_29_min'] = df_19['ACS19_5yr_B08134076']
# Public Transportation (Excluding Taxicab): Bus: 30 To 34 Minutes,ACS19_5yr_B08134077
df['ttl_bus_wrkr_cmute_30_to_34_min'] = df_19['ACS19_5yr_B08134077']
# Public Transportation (Excluding Taxicab): Bus: 35 To 44 Minutes,ACS19_5yr_B08134078
df['ttl_bus_wrkr_cmute_35_to_44_min'] = df_19['ACS19_5yr_B08134078']
# Public Transportation (Excluding Taxicab): Bus: 45 To 59 Minutes,ACS19_5yr_B08134079
df['ttl_bus_wrkr_cmute_45_to_59_min'] = df_19['ACS19_5yr_B08134079']
# Public Transportation (Excluding Taxicab): Bus: 60 Or More Minutes,ACS19_5yr_B08134080
df['ttl_bus_wrkr_cmute_over_60_min'] = df_19['ACS19_5yr_B08134080']
# 15 min range Percents
df['pct_bus_wrkr_cmute'] = df['ttl_bus_wrkr_cmute'] / df['ttl_wrkr_cmute']
df['pct_bus_wrkr_cmute_01_to_14_min'] = (df['ttl_bus_wrkr_cmute_under_10_min'] + df['ttl_bus_wrkr_cmute_10_to_14_min']) / df['ttl_wrkr_cmute']
df['pct_bus_wrkr_cmute_15_to_29_min'] = (df['ttl_bus_wrkr_cmute_15_to_14_min'] + df['ttl_bus_wrkr_cmute_20_to_24_min'] + df['ttl_bus_wrkr_cmute_25_to_29_min']) / df['ttl_wrkr_cmute']
df['pct_bus_wrkr_cmute_30_to_44_min'] = (df['ttl_bus_wrkr_cmute_30_to_34_min'] + df['ttl_bus_wrkr_cmute_35_to_44_min']) / df['ttl_wrkr_cmute']
df['pct_bus_wrkr_cmute_44_to_59_min'] = df['ttl_bus_wrkr_cmute_45_to_59_min'] / df['ttl_wrkr_cmute']
df['pct_bus_wrkr_cmute_over_60_min'] = df['ttl_bus_wrkr_cmute_over_60_min'] / df['ttl_wrkr_cmute']
# # 15 min range Percents
# df['pct_bus_wrkr_cmute_01_to_14_min'] = (df['ttl_bus_wrkr_cmute_under_10_min'] + df['ttl_bus_wrkr_cmute_10_to_14_min']) / df['ttl_bus_wrkr_cmute']
# df['pct_bus_wrkr_cmute_15_to_29_min'] = (df['ttl_bus_wrkr_cmute_15_to_14_min'] + df['ttl_bus_wrkr_cmute_20_to_24_min'] + df['ttl_bus_wrkr_cmute_25_to_29_min']) / df['ttl_bus_wrkr_cmute']
# df['pct_bus_wrkr_cmute_30_to_44_min'] = (df['ttl_bus_wrkr_cmute_30_to_34_min'] + df['ttl_bus_wrkr_cmute_35_to_44_min']) / df['ttl_bus_wrkr_cmute']
# df['pct_bus_wrkr_cmute_44_to_59_min'] = df['ttl_bus_wrkr_cmute_45_to_59_min'] / df['ttl_bus_wrkr_cmute']
# df['pct_bus_wrkr_cmute_over_60_min'] = df['ttl_bus_wrkr_cmute_over_60_min'] / df['ttl_bus_wrkr_cmute']
# Zero Check
df['zero_check'] = df['pct_bus_wrkr_cmute_01_to_14_min'] * \
                        df['pct_bus_wrkr_cmute_15_to_29_min'] * \
                        df['pct_bus_wrkr_cmute_30_to_44_min'] * \
                        df['pct_bus_wrkr_cmute_44_to_59_min'] * \
                        df['pct_bus_wrkr_cmute_over_60_min'] * df['zero_check']


#  Subway Or Elevated Rail Light Rail Streetcar Or Trolley (Carro P̼blico In Puerto Rico),ACS19_5yr_B08134081
df['ttl_local_rails_wrkr_cmute'] = df_19['ACS19_5yr_B08134081']
#  Subway Or Elevated Rail Light Rail Streetcar Or Trolley (Carro P̼blico In Puerto Rico): Less Than 10 Minutes,ACS19_5yr_B08134082
df['ttl_local_rails_wrkr_cmute_under_10_min'] = df_19['ACS19_5yr_B08134082']
#  Subway Or Elevated Rail Light Rail Streetcar Or Trolley (Carro P̼blico In Puerto Rico): 10 To 14 Minutes,ACS19_5yr_B08134083
df['ttl_local_rails_wrkr_cmute_10_to_14_min'] = df_19['ACS19_5yr_B08134083']
#  Subway Or Elevated Rail Light Rail Streetcar Or Trolley (Carro P̼blico In Puerto Rico): 15 To 19 Minutes,ACS19_5yr_B08134084
df['ttl_local_rails_wrkr_cmute_15_to_14_min'] = df_19['ACS19_5yr_B08134084']
#  Subway Or Elevated Rail Light Rail Streetcar Or Trolley (Carro P̼blico In Puerto Rico): 20 To 24 Minutes,ACS19_5yr_B08134085
df['ttl_local_rails_wrkr_cmute_20_to_24_min'] = df_19['ACS19_5yr_B08134085']
#  Subway Or Elevated Rail Light Rail Streetcar Or Trolley (Carro P̼blico In Puerto Rico): 25 To 29 Minutes,ACS19_5yr_B08134086
df['ttl_local_rails_wrkr_cmute_25_to_29_min'] = df_19['ACS19_5yr_B08134086']
#  Subway Or Elevated Rail Light Rail Streetcar Or Trolley (Carro P̼blico In Puerto Rico): 30 To 34 Minutes,ACS19_5yr_B08134087
df['ttl_local_rails_wrkr_cmute_30_to_34_min'] = df_19['ACS19_5yr_B08134087']
#  Subway Or Elevated Rail Light Rail Streetcar Or Trolley (Carro P̼blico In Puerto Rico): 35 To 44 Minutes,ACS19_5yr_B08134088
df['ttl_local_rails_wrkr_cmute_35_to_44_min'] = df_19['ACS19_5yr_B08134088']
#  Subway Or Elevated Rail Light Rail Streetcar Or Trolley (Carro P̼blico In Puerto Rico): 45 To 59 Minutes,ACS19_5yr_B08134089
df['ttl_local_rails_wrkr_cmute_45_to_59_min'] = df_19['ACS19_5yr_B08134089']
#  Subway Or Elevated Rail Light Rail Streetcar Or Trolley (Carro P̼blico In Puerto Rico): 60 Or More Minutes,ACS19_5yr_B08134090
df['ttl_local_rails_wrkr_cmute_over_60_min'] = df_19['ACS19_5yr_B08134090']
# 15 min range Percents
df['pct_local_rails_wrkr_cmute'] = df['ttl_local_rails_wrkr_cmute'] / df['ttl_wrkr_cmute']
df['pct_local_rails_wrkr_cmute_01_to_14_min'] = (df['ttl_local_rails_wrkr_cmute_under_10_min'] + df['ttl_local_rails_wrkr_cmute_10_to_14_min']) / df['ttl_wrkr_cmute']
df['pct_local_rails_wrkr_cmute_15_to_29_min'] = (df['ttl_local_rails_wrkr_cmute_15_to_14_min'] + df['ttl_local_rails_wrkr_cmute_20_to_24_min'] + df['ttl_local_rails_wrkr_cmute_25_to_29_min']) / df['ttl_wrkr_cmute']
df['pct_local_rails_wrkr_cmute_30_to_44_min'] = (df['ttl_local_rails_wrkr_cmute_30_to_34_min'] + df['ttl_local_rails_wrkr_cmute_35_to_44_min']) / df['ttl_wrkr_cmute']
df['pct_local_rails_wrkr_cmute_44_to_59_min'] = df['ttl_local_rails_wrkr_cmute_45_to_59_min'] / df['ttl_wrkr_cmute']
df['pct_local_rails_wrkr_cmute_over_60_min'] = df['ttl_local_rails_wrkr_cmute_over_60_min'] / df['ttl_wrkr_cmute']
# # 15 min range Percents
# df['pct_local_rails_wrkr_cmute_01_to_14_min'] = (df['ttl_local_rails_wrkr_cmute_under_10_min'] + df['ttl_local_rails_wrkr_cmute_10_to_14_min']) / df['ttl_local_rails_wrkr_cmute']
# df['pct_local_rails_wrkr_cmute_15_to_29_min'] = (df['ttl_local_rails_wrkr_cmute_15_to_14_min'] + df['ttl_local_rails_wrkr_cmute_20_to_24_min'] + df['ttl_local_rails_wrkr_cmute_25_to_29_min']) / df['ttl_local_rails_wrkr_cmute']
# df['pct_local_rails_wrkr_cmute_30_to_44_min'] = (df['ttl_local_rails_wrkr_cmute_30_to_34_min'] + df['ttl_local_rails_wrkr_cmute_35_to_44_min']) / df['ttl_local_rails_wrkr_cmute']
# df['pct_local_rails_wrkr_cmute_44_to_59_min'] = df['ttl_local_rails_wrkr_cmute_45_to_59_min'] / df['ttl_local_rails_wrkr_cmute']
# df['pct_local_rails_wrkr_cmute_over_60_min'] = df['ttl_local_rails_wrkr_cmute_over_60_min'] / df['ttl_local_rails_wrkr_cmute']
# Zero Check
df['zero_check'] = df['pct_local_rails_wrkr_cmute_01_to_14_min'] * \
                        df['pct_local_rails_wrkr_cmute_15_to_29_min'] * \
                        df['pct_local_rails_wrkr_cmute_30_to_44_min'] * \
                        df['pct_local_rails_wrkr_cmute_44_to_59_min'] * \
                        df['pct_local_rails_wrkr_cmute_over_60_min'] * df['zero_check']


#  Long-Distance Train Or Commuter Rail Or Ferryboat,ACS19_5yr_B08134091
df['ttl_long_distance_public_transport_wrkr_cmute'] = df_19['ACS19_5yr_B08134091']
#  Long-Distance Train Or Commuter Rail Or Ferryboat: Less Than 10 Minutes,ACS19_5yr_B08134092
df['ttl_long_distance_public_transport_wrkr_cmute_under_10_min'] = df_19['ACS19_5yr_B08134092']
#  Long-Distance Train Or Commuter Rail Or Ferryboat: 10 To 14 Minutes,ACS19_5yr_B08134093
df['ttl_long_distance_public_transport_wrkr_cmute_10_to_14_min'] = df_19['ACS19_5yr_B08134093']
#  Long-Distance Train Or Commuter Rail Or Ferryboat: 15 To 19 Minutes,ACS19_5yr_B08134094
df['ttl_long_distance_public_transport_wrkr_cmute_15_to_14_min'] = df_19['ACS19_5yr_B08134094']
#  Long-Distance Train Or Commuter Rail Or Ferryboat: 20 To 24 Minutes,ACS19_5yr_B08134095
df['ttl_long_distance_public_transport_wrkr_cmute_20_to_24_min'] = df_19['ACS19_5yr_B08134095']
#  Long-Distance Train Or Commuter Rail Or Ferryboat: 25 To 29 Minutes,ACS19_5yr_B08134096
df['ttl_long_distance_public_transport_wrkr_cmute_25_to_29_min'] = df_19['ACS19_5yr_B08134096']
#  Long-Distance Train Or Commuter Rail Or Ferryboat: 30 To 34 Minutes,ACS19_5yr_B08134097
df['ttl_long_distance_public_transport_wrkr_cmute_30_to_34_min'] = df_19['ACS19_5yr_B08134097']
#  Long-Distance Train Or Commuter Rail Or Ferryboat: 35 To 44 Minutes,ACS19_5yr_B08134098
df['ttl_long_distance_public_transport_wrkr_cmute_35_to_44_min'] = df_19['ACS19_5yr_B08134098']
#  Long-Distance Train Or Commuter Rail Or Ferryboat: 45 To 59 Minutes,ACS19_5yr_B08134099
df['ttl_long_distance_public_transport_wrkr_cmute_45_to_59_min'] = df_19['ACS19_5yr_B08134099']
#  Long-Distance Train Or Commuter Rail Or Ferryboat: 60 Or More Minutes,ACS19_5yr_B08134100
df['ttl_long_distance_public_transport_wrkr_cmute_over_60_min'] = df_19['ACS19_5yr_B08134100']
# 15 min range Percents
df['pct_long_distance_public_transport_wrkr_cmute'] = df['ttl_long_distance_public_transport_wrkr_cmute'] / df['ttl_wrkr_cmute']
df['pct_long_distance_public_transport_wrkr_cmute_01_to_14_min'] = (df['ttl_long_distance_public_transport_wrkr_cmute_under_10_min'] + df['ttl_long_distance_public_transport_wrkr_cmute_10_to_14_min']) / df['ttl_wrkr_cmute']
df['pct_long_distance_public_transport_wrkr_cmute_15_to_29_min'] = (df['ttl_long_distance_public_transport_wrkr_cmute_15_to_14_min'] + df['ttl_long_distance_public_transport_wrkr_cmute_20_to_24_min'] + df['ttl_long_distance_public_transport_wrkr_cmute_25_to_29_min']) / df['ttl_wrkr_cmute']
df['pct_long_distance_public_transport_wrkr_cmute_30_to_44_min'] = (df['ttl_long_distance_public_transport_wrkr_cmute_30_to_34_min'] + df['ttl_long_distance_public_transport_wrkr_cmute_35_to_44_min']) / df['ttl_wrkr_cmute']
df['pct_long_distance_public_transport_wrkr_cmute_44_to_59_min'] = df['ttl_long_distance_public_transport_wrkr_cmute_45_to_59_min'] / df['ttl_wrkr_cmute']
df['pct_long_distance_public_transport_wrkr_cmute_over_60_min'] = df['ttl_long_distance_public_transport_wrkr_cmute_over_60_min'] / df['ttl_wrkr_cmute']
# # 15 min range Percents
# df['pct_long_distance_public_transport_wrkr_cmute_01_to_14_min'] = (df['ttl_long_distance_public_transport_wrkr_cmute_under_10_min'] + df['ttl_long_distance_public_transport_wrkr_cmute_10_to_14_min']) / df['ttl_long_distance_public_transport_wrkr_cmute']
# df['pct_long_distance_public_transport_wrkr_cmute_15_to_29_min'] = (df['ttl_long_distance_public_transport_wrkr_cmute_15_to_14_min'] + df['ttl_long_distance_public_transport_wrkr_cmute_20_to_24_min'] + df['ttl_long_distance_public_transport_wrkr_cmute_25_to_29_min']) / df['ttl_long_distance_public_transport_wrkr_cmute']
# df['pct_long_distance_public_transport_wrkr_cmute_30_to_44_min'] = (df['ttl_long_distance_public_transport_wrkr_cmute_30_to_34_min'] + df['ttl_long_distance_public_transport_wrkr_cmute_35_to_44_min']) / df['ttl_long_distance_public_transport_wrkr_cmute']
# df['pct_long_distance_public_transport_wrkr_cmute_44_to_59_min'] = df['ttl_long_distance_public_transport_wrkr_cmute_45_to_59_min'] / df['ttl_long_distance_public_transport_wrkr_cmute']
# df['pct_long_distance_public_transport_wrkr_cmute_over_60_min'] = df['ttl_long_distance_public_transport_wrkr_cmute_over_60_min'] / df['ttl_long_distance_public_transport_wrkr_cmute']
# Zero Check
df['zero_check'] = df['pct_long_distance_public_transport_wrkr_cmute_01_to_14_min'] * \
                        df['pct_long_distance_public_transport_wrkr_cmute_15_to_29_min'] * \
                        df['pct_long_distance_public_transport_wrkr_cmute_30_to_44_min'] * \
                        df['pct_long_distance_public_transport_wrkr_cmute_44_to_59_min'] * \
                        df['pct_long_distance_public_transport_wrkr_cmute_over_60_min'] * df['zero_check']


# Walked.20,ACS19_5yr_B08134101
df['ttl_walked_wrkr_cmute'] = df_19['ACS19_5yr_B08134101']
# Walked: Less Than 10 Minutes,ACS19_5yr_B08134102
df['ttl_walked_wrkr_cmute_under_10_min'] = df_19['ACS19_5yr_B08134102']
# Walked: 10 To 14 Minutes,ACS19_5yr_B08134103
df['ttl_walked_wrkr_cmute_10_to_14_min'] = df_19['ACS19_5yr_B08134103']
# Walked: 15 To 19 Minutes,ACS19_5yr_B08134104
df['ttl_walked_wrkr_cmute_15_to_14_min'] = df_19['ACS19_5yr_B08134104']
# Walked: 20 To 24 Minutes,ACS19_5yr_B08134105
df['ttl_walked_wrkr_cmute_20_to_24_min'] = df_19['ACS19_5yr_B08134105']
# Walked: 25 To 29 Minutes,ACS19_5yr_B08134106
df['ttl_walked_wrkr_cmute_25_to_29_min'] = df_19['ACS19_5yr_B08134106']
# Walked: 30 To 34 Minutes,ACS19_5yr_B08134107
df['ttl_walked_wrkr_cmute_30_to_34_min'] = df_19['ACS19_5yr_B08134107']
# Walked: 35 To 44 Minutes,ACS19_5yr_B08134108
df['ttl_walked_wrkr_cmute_35_to_44_min'] = df_19['ACS19_5yr_B08134108']
# Walked: 45 To 59 Minutes,ACS19_5yr_B08134109
df['ttl_walked_wrkr_cmute_45_to_59_min'] = df_19['ACS19_5yr_B08134109']
# Walked: 60 Or More Minutes,ACS19_5yr_B08134110
df['ttl_walked_wrkr_cmute_over_60_min'] = df_19['ACS19_5yr_B08134110']
# 15 min range Percents
df['pct_walked_wrkr_cmute'] = df['ttl_walked_wrkr_cmute'] / df['ttl_wrkr_cmute']
df['pct_walked_wrkr_cmute_01_to_14_min'] = (df['ttl_walked_wrkr_cmute_under_10_min'] + df['ttl_walked_wrkr_cmute_10_to_14_min']) / df['ttl_wrkr_cmute']
df['pct_walked_wrkr_cmute_15_to_29_min'] = (df['ttl_walked_wrkr_cmute_15_to_14_min'] + df['ttl_walked_wrkr_cmute_20_to_24_min'] + df['ttl_walked_wrkr_cmute_25_to_29_min']) / df['ttl_wrkr_cmute']
df['pct_walked_wrkr_cmute_30_to_44_min'] = (df['ttl_walked_wrkr_cmute_30_to_34_min'] + df['ttl_walked_wrkr_cmute_35_to_44_min']) / df['ttl_wrkr_cmute']
df['pct_walked_wrkr_cmute_44_to_59_min'] = df['ttl_walked_wrkr_cmute_45_to_59_min'] / df['ttl_wrkr_cmute']
df['pct_walked_wrkr_cmute_over_60_min'] = df['ttl_walked_wrkr_cmute_over_60_min'] / df['ttl_wrkr_cmute']
# # 15 min range Percents
# df['pct_walked_wrkr_cmute_01_to_14_min'] = (df['ttl_walked_wrkr_cmute_under_10_min'] + df['ttl_walked_wrkr_cmute_10_to_14_min']) / df['ttl_walked_wrkr_cmute']
# df['pct_walked_wrkr_cmute_15_to_29_min'] = (df['ttl_walked_wrkr_cmute_15_to_14_min'] + df['ttl_walked_wrkr_cmute_20_to_24_min'] + df['ttl_walked_wrkr_cmute_25_to_29_min']) / df['ttl_walked_wrkr_cmute']
# df['pct_walked_wrkr_cmute_30_to_44_min'] = (df['ttl_walked_wrkr_cmute_30_to_34_min'] + df['ttl_walked_wrkr_cmute_35_to_44_min']) / df['ttl_walked_wrkr_cmute']
# df['pct_walked_wrkr_cmute_44_to_59_min'] = df['ttl_walked_wrkr_cmute_45_to_59_min'] / df['ttl_walked_wrkr_cmute']
# df['pct_walked_wrkr_cmute_over_60_min'] = df['ttl_walked_wrkr_cmute_over_60_min'] / df['ttl_walked_wrkr_cmute']
# Zero Check
df['zero_check'] = df['pct_walked_wrkr_cmute_01_to_14_min'] * \
                        df['pct_walked_wrkr_cmute_15_to_29_min'] * \
                        df['pct_walked_wrkr_cmute_30_to_44_min'] * \
                        df['pct_walked_wrkr_cmute_44_to_59_min'] * \
                        df['pct_walked_wrkr_cmute_over_60_min'] * df['zero_check']

# Taxicab Motorcycle Bicycle Or Other Means.19,ACS19_5yr_B08134111
df['ttl_misc_trspt_method_wrkr_cmute'] = df_19['ACS19_5yr_B08134111']
# Taxicab Motorcycle Bicycle Or Other Means: Less Than 10 Minutes,ACS19_5yr_B08134112
df['ttl_misc_trspt_method_wrkr_cmute_under_10_min'] = df_19['ACS19_5yr_B08134112']
# Taxicab Motorcycle Bicycle Or Other Means: 10 To 14 Minutes,ACS19_5yr_B08134113
df['ttl_misc_trspt_method_wrkr_cmute_10_to_14_min'] = df_19['ACS19_5yr_B08134113']
# Taxicab Motorcycle Bicycle Or Other Means: 15 To 19 Minutes,ACS19_5yr_B08134114
df['ttl_misc_trspt_method_wrkr_cmute_15_to_14_min'] = df_19['ACS19_5yr_B08134114']
# Taxicab Motorcycle Bicycle Or Other Means: 20 To 24 Minutes,ACS19_5yr_B08134115
df['ttl_misc_trspt_method_wrkr_cmute_20_to_24_min'] = df_19['ACS19_5yr_B08134115']
# Taxicab Motorcycle Bicycle Or Other Means: 25 To 29 Minutes,ACS19_5yr_B08134116
df['ttl_misc_trspt_method_wrkr_cmute_25_to_29_min'] = df_19['ACS19_5yr_B08134116']
# Taxicab Motorcycle Bicycle Or Other Means: 30 To 34 Minutes,ACS19_5yr_B08134117
df['ttl_misc_trspt_method_wrkr_cmute_30_to_34_min'] = df_19['ACS19_5yr_B08134117']
# Taxicab Motorcycle Bicycle Or Other Means: 35 To 44 Minutes,ACS19_5yr_B08134118
df['ttl_misc_trspt_method_wrkr_cmute_35_to_44_min'] = df_19['ACS19_5yr_B08134118']
# Taxicab Motorcycle Bicycle Or Other Means: 45 To 59 Minutes,ACS19_5yr_B08134119
df['ttl_misc_trspt_method_wrkr_cmute_45_to_59_min'] = df_19['ACS19_5yr_B08134119']
# Taxicab Motorcycle Bicycle Or Other Means: 60 Or More Minutes,ACS19_5yr_B08134120
df['ttl_misc_trspt_method_wrkr_cmute_over_60_min'] = df_19['ACS19_5yr_B08134120']
# 15 min range Percents
df['pct_misc_trspt_method_wrkr_cmute'] = df['ttl_misc_trspt_method_wrkr_cmute'] / df['ttl_wrkr_cmute']
df['pct_misc_trspt_method_wrkr_cmute_01_to_14_min'] = (df['ttl_misc_trspt_method_wrkr_cmute_under_10_min'] + df['ttl_misc_trspt_method_wrkr_cmute_10_to_14_min']) / df['ttl_wrkr_cmute']
df['pct_misc_trspt_method_wrkr_cmute_15_to_29_min'] = (df['ttl_misc_trspt_method_wrkr_cmute_15_to_14_min'] + df['ttl_misc_trspt_method_wrkr_cmute_20_to_24_min'] + df['ttl_misc_trspt_method_wrkr_cmute_25_to_29_min']) / df['ttl_wrkr_cmute']
df['pct_misc_trspt_method_wrkr_cmute_30_to_44_min'] = (df['ttl_misc_trspt_method_wrkr_cmute_30_to_34_min'] + df['ttl_misc_trspt_method_wrkr_cmute_35_to_44_min']) / df['ttl_wrkr_cmute']
df['pct_misc_trspt_method_wrkr_cmute_44_to_59_min'] = df['ttl_misc_trspt_method_wrkr_cmute_45_to_59_min'] / df['ttl_wrkr_cmute']
df['pct_misc_trspt_method_wrkr_cmute_over_60_min'] = df['ttl_misc_trspt_method_wrkr_cmute_over_60_min'] / df['ttl_wrkr_cmute']
# # 15 min range Percents
# df['pct_misc_trspt_method_wrkr_cmute_01_to_14_min'] = (df['ttl_misc_trspt_method_wrkr_cmute_under_10_min'] + df['ttl_misc_trspt_method_wrkr_cmute_10_to_14_min']) / df['ttl_misc_trspt_method_wrkr_cmute']
# df['pct_misc_trspt_method_wrkr_cmute_15_to_29_min'] = (df['ttl_misc_trspt_method_wrkr_cmute_15_to_14_min'] + df['ttl_misc_trspt_method_wrkr_cmute_20_to_24_min'] + df['ttl_misc_trspt_method_wrkr_cmute_25_to_29_min']) / df['ttl_misc_trspt_method_wrkr_cmute']
# df['pct_misc_trspt_method_wrkr_cmute_30_to_44_min'] = (df['ttl_misc_trspt_method_wrkr_cmute_30_to_34_min'] + df['ttl_misc_trspt_method_wrkr_cmute_35_to_44_min']) / df['ttl_misc_trspt_method_wrkr_cmute']
# df['pct_misc_trspt_method_wrkr_cmute_44_to_59_min'] = df['ttl_misc_trspt_method_wrkr_cmute_45_to_59_min'] / df['ttl_misc_trspt_method_wrkr_cmute']
# df['pct_misc_trspt_method_wrkr_cmute_over_60_min'] = df_19['ACS19_5yr_B08134120'] / df['ttl_misc_trspt_method_wrkr_cmute']


# df = df[df['zero_check'] !=0]
# df = df.loc[(df[['zero_check']] != 0).all(axis=1)]
# df = df.dropna(subset=['zero_check', 'aggregate_time_commute', 'aggregate_time_commute_drive_alone', 'aggregate_time_commute_carpool', 'aggregate_time_commute_public_transit', 'aggregate_time_commute_other_means'])

##### HOUSEHOLD DATA
df['ttl_households'] = df_19['ACS19_5yr_B11001001']

##### UNEMPLOYMENT DATA
# # Population 16 Years and Over: in Labor Force - ACS19_5yr_B23025002
# df['population_in_labor_force'] = df_19['ACS19_5yr_B23025002']
# Population 16 Years and Over: in Labor Force: Civilian Labor Force - ACS19_5yr_B23025003
df['population_in_labor_force'] = df_19['ACS19_5yr_B23025003']
# Population 16 Years and Over: in Labor Force: Civilian Labor Force: Employed - ACS19_5yr_B23025004
df['population_employed'] = df_19['ACS19_5yr_B23025004']
# Population 16 Years and Over: in Labor Force: Civilian Labor Force: Unemployed - ACS19_5yr_B23025005
df['population_unemployed'] = df_19['ACS19_5yr_B23025005']
df['pct_unemployed'] = df['population_unemployed'] / df['population_in_labor_force'] * 100

df['population_density'] = df['total_population'] / df['land_area']# / 1000000 # convert to sqkm
df['worker_density'] = df['total_workers'] / df['land_area']# / 1000000 # convert to sqkm
df['vehicles_density'] = df['aggregate_num_vehicles'] / df['land_area']# / 1000000 # convert to sqkm
df['vehicles_per_worker'] = df['aggregate_num_vehicles'] / df['total_workers']
df['log_vehicles_per_worker'] = np.log(df['vehicles_per_worker'])
df['vehicles_per_worker_x_landarea'] = (df['aggregate_num_vehicles'] / df['total_workers']) * df['land_area'] # / 1000000 # convert to sqkm
df['vehicles_per_worker_per_landarea'] = (df['aggregate_num_vehicles'] / df['total_workers']) / df['land_area'] # / 1000000 # convert to sqkm
df['vehicles_x_worker_per_landarea'] = (df['aggregate_num_vehicles'] * df['total_workers']) / df['land_area'] # / 1000000 # convert to sqkm
df['vehicle_density_x_vehicles_per_worker'] = (df['vehicles_density'] * df['vehicles_per_worker'])

df['average_commute_time_per_worker_drive_alone'] = df['aggregate_time_commute_drive_alone'] / df['total_workers_drove_alone']
average_commute_time_per_worker_drive_alone = sum(df['aggregate_time_commute_drive_alone']) / sum(df['total_workers_drove_alone'])

df['average_commute_time_per_worker_carpool'] = df['aggregate_time_commute_carpool'] / df['total_workers_carpooled']
average_commute_time_per_worker_carpool = sum(df['aggregate_time_commute_carpool']) / sum(df['total_workers_carpooled'])

df['average_commute_time_per_worker_public_transit'] = df['aggregate_time_commute_public_transit'] / df['total_workers_public_transit']
average_commute_time_per_worker_public_transit = sum(df['aggregate_time_commute_public_transit']) / sum(df['total_workers_public_transit'])

df['average_commute_time_per_worker_other_means'] = df['aggregate_time_commute_other_means'] / df['total_workers_other_means']
average_commute_time_per_worker_other_means = sum(df['aggregate_time_commute_other_means']) / sum(df['total_workers_other_means'])



##### vehicles_data_cleanup
df['households_with_0_vehicle'] = df_19['ACS19_5yr_B08014002']
df['households_with_1_vehicle'] = df_19['ACS19_5yr_B08014003']
df['households_with_2_vehicle'] = df_19['ACS19_5yr_B08014004']
df['households_with_3_vehicle'] = df_19['ACS19_5yr_B08014005']
df['households_with_4_vehicle'] = df_19['ACS19_5yr_B08014006']
df['households_with_5_or_more_vehicle'] = df_19['ACS19_5yr_B08014007']



c:\Users\cjong\anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\cjong\anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\cjong\anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\cjong\anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\cjong\anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\cjong\anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: divide by zero encountered in l

In [5]:
df.to_csv('./processed_data/american_community_survey_data_2019.csv')
